In [ ]:
import cv2
import numpy as np
import os
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.io import output_notebook

def cvt2bokeh(img):
    h, w = img.shape[:2]
    container = np.empty((h, w), dtype=np.uint32)
    view = container.view(dtype=np.uint8).reshape((h, w, 4))
    view[:, :, 0] = img[::-1, :, 0]  # copy red channel
    view[:, :, 1] = img[::-1, :, 1]  # copy blue channel
    view[:, :, 2] = img[::-1, :, 2]  # copy green channel
    view[:, :, 3] = 255
    return container

def fundus_roi(image, mask=None):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv_image)
    threshold = 15
    _, roi = cv2.threshold(v, threshold, 1, cv2.THRESH_BINARY)
    roi = roi.astype(bool)
    white_pixels = np.argwhere(roi == 1)
    if white_pixels.size == 0:
        print("Aucun pixel blanc trouvé dans le masque.")
        return {"roi": roi, "diameter": 0, "image": image}
    x_min, y_min = np.min(white_pixels, axis=0)
    x_max, y_max = np.max(white_pixels, axis=0)
    diameter_x = x_max - x_min
    diameter_y = y_max - y_min
    diameter = int(np.maximum(diameter_x, diameter_y))
    return {"roi": roi, "diameter": diameter, "image": image}

def apply_seoud(image, roi=None, diameter=None):
    data = fundus_roi(image)
    roi = data["roi"]
    roi = roi.astype(np.uint8)
    D = data["diameter"]
    roi_bool = roi > 0

    mean_img = np.mean(image[roi_bool])
    std_img = np.std(image[roi_bool])
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))
    dilated_image = cv2.dilate(image, kernel, iterations=25)
    image[roi == 0] = np.mean(image, axis=(0, 1))
    roi = cv2.erode(roi, np.ones((9, 9)), iterations=1)
    expended_roi = np.expand_dims(roi, 2)
    dilated_image[dilated_image[:, :, 0] < 100] = np.mean(dilated_image, axis=(0, 1))

    image = image * expended_roi + (1 - expended_roi) * dilated_image

    h0 = D // 5
    h1 = D // 10
    h2 = D // 360
    h3 = D // 28
    h0 += (h0 + 1) % 2
    h1 += (h1 + 1) % 2
    h2 += (h2 + 1) % 2
    h3 += (h3 + 1) % 2

    blur = cv2.medianBlur(image, min(h1, 255))

    std = local_std(image, h1)
    std = std.clip(std_img, std.max())

    c1s = cv2.split(image)
    c2s = cv2.split(blur)
    out = []
    for c1, c2 in zip(c1s, c2s):
        out.append(cv2.addWeighted(c1, 1, c2, -1, np.mean(image)))
    output = cv2.merge(out)

    hpf = cv2.addWeighted(output, 1, cv2.blur(output, (h3, h3)), -1, np.mean(output))

    output = cv2.addWeighted(output.astype(np.float32), 1, cv2.divide(hpf.astype(np.float32), std + 1e-7), 1, 0).clip(0, 255)

    cs = cv2.split(output)
    out = []
    for c in cs:
        c = (std_img * (c - c.mean()) / c.std()) + 127
        c = c.clip(mean_img - 3 * std_img, mean_img + 3 * std_img)
        out.append(c)

    output = cv2.merge(out).clip(0, 255).astype(np.uint8)
    return {"image": image, "output": output * expended_roi}

def local_std(image, size):
    img_sq = cv2.multiply(image.astype(np.float32), image.astype(np.float32))
    E_img_sq = cv2.blur(img_sq, (size, size))

    E_local = cv2.blur(image.astype(np.float32), (size, size))
    sq_E_local = cv2.multiply(E_local, E_local)

    var = cv2.add(E_img_sq, -sq_E_local)
    std = cv2.sqrt(var.clip(1e-3, var.max()))
    return std

# Charger une image depuis le dossier ../data
image_path = "../data/image_1.jpg"
image = cv2.imread(image_path)

if image is None:
    print(f"Erreur: Impossible de charger l'image {image_path}.")
else:
    output_notebook()  # Active la sortie Bokeh dans le notebook

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    seoud_image = apply_seoud(image)

    # Convertir les images en format Bokeh RGBA avec cvt2bokeh
    image_bokeh = cvt2bokeh(image_rgb)
    seoud_bokeh = cvt2bokeh(seoud_image['output'])


    # Créer les figures Bokeh pour afficher les images
    p1 = figure(title="Image originale", 
                x_range=(0, image_rgb.shape[1]), y_range=(0, image_rgb.shape[0]),
                width=image_rgb.shape[1]//6, height=image_rgb.shape[0]//6)
    p1.image_rgba(image=[image_bokeh], x=0, y=0, dw=image_rgb.shape[1], dh=image_rgb.shape[0])

    p2 = figure(title="Image Seoud RGB",
                x_range=p1.x_range, y_range=p1.y_range,
                width=image_rgb.shape[1]//6, height=image_rgb.shape[0]//6)
    p2.image_rgba(image=[seoud_bokeh], x=0, y=0, dw=seoud_image['output'].shape[1], dh=seoud_image['output'].shape[0])

    # Créer une grille de plots Bokeh
    grid = gridplot([[p1, p2]], toolbar_location='right')

    # Afficher la grille de plots dans le notebook
    show(grid)
